<a href="https://colab.research.google.com/github/Mahaveersalvih/movie-recommendation-/blob/main/Copy_of_Project_18_Mood_based_Movie_Recommendation_System_using_Machine_Learning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the dependencies

In [ ]:
!pip install streamlit
import numpy as np
import pandas as pd
import streamlit as st
import requests
import pickle
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# loading the data from the csv file to apandas dataframe
movies_data = pd.read_csv('/content/movies.csv')

In [ ]:
# printing the first 5 rows of the dataframe
movies_data.head()

In [ ]:
# number of rows and columns in the data frame

movies_data.shape

In [ ]:
# selecting the relevant features for recommendation

selected_features = ['genres','keywords','tagline','cast','director','id']
print(selected_features)

In [ ]:
# replacing the null valuess with null string

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [ ]:
# combining all the 5 selected features

combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']+' '+movies_data['id'].astype(str)

In [ ]:
print(combined_features)

In [ ]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [ ]:
print(feature_vectors)

Cosine Similarity

In [ ]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

In [ ]:
print(similarity.shape)

Getting the movie name from the user

In [ ]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

In [ ]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

In [ ]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

In [ ]:
close_match = find_close_match[0]
print(close_match)

In [ ]:
# finding the index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

In [ ]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

In [ ]:
len(similarity_score)

In [ ]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

In [ ]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movie Recommendation Sytem

In [ ]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<20):
    print(i, '.',title_from_index)
    i+=1

In [ ]:
import pickle
with open('movies.pkl','wb')as file:
    pickle.dump(movies_data,file)

with open('similarity.pkl','wb')as file:
    pickle.dump(similarity,file)

In [ ]:
mood_to_genres = {
    'happy': ['Comedy', 'Family', 'Animation', 'Adventure'],
    'sad': ['Drama', 'Tragedy', 'Romance'],
    'action': ['Action', 'Adventure', 'Science Fiction', 'Thriller'],
    'romantic': ['Romance', 'Drama'],
    'thrilling': ['Thriller', 'Action', 'Crime', 'Mystery'],
    'family-friendly': ['Family', 'Animation', 'Comedy', 'Adventure'],
    'dark': ['Thriller', 'Crime', 'Mystery', 'Horror', 'Science Fiction'],
    'funny': ['Comedy', 'Animation', 'Family']
}

print("Mood to genre mapping dictionary created:")
print(mood_to_genres)

In [ ]:
user_mood = input("Enter your current mood (e.g., happy, sad, action, romantic, thrilling, family-friendly, dark, funny): ")
print(f"You entered: {user_mood}")

In [ ]:
genres_for_mood = mood_to_genres.get(user_mood.lower(), [])

if not genres_for_mood:
    print(f"Mood '{user_mood}' not recognized. Please choose from: {', '.join(mood_to_genres.keys())}.")
    print("No genres retrieved for this mood.")
else:
    print(f"Genres associated with '{user_mood}': {genres_for_mood}")

In [ ]:
import pandas as pd
import random

movies_data = pd.read_csv('/content/movies.csv')
movies_data['genres'] = movies_data['genres'].fillna('') # Fill NaN values with empty strings
mood_based_recommendations = []

if genres_for_mood:
    for index, row in movies_data.iterrows():
        movie_genres = row['genres'].split(' ')
        if any(genre in movie_genres for genre in genres_for_mood):
            mood_based_recommendations.append(index)

# Shuffle the recommendations to get different movies each time
random.shuffle(mood_based_recommendations)

print(f"Total movies found for '{user_mood}': {len(mood_based_recommendations)}")
print("First 10 recommended movie indices for this mood (shuffled):")
for i, movie_index in enumerate(mood_based_recommendations[:10]):
    title = movies_data.loc[movie_index, 'title']
    print(f"  {i+1}. {title} (Index: {movie_index})")

In [ ]:
print(f"\nTop 15 Movies for your '{user_mood}' mood:")

if mood_based_recommendations:
    for i, movie_index in enumerate(mood_based_recommendations[:15]):
        title = movies_data.loc[movie_index, 'title']
        print(f"  {i+1}. {title}")
else:
    print("No movies found for this mood.")

In [ ]:
# 1. Install Streamlit
!pip install -q streamlit

# 2. Get your public IP (You will need this password shortly)
!wget -q -O - ipv4.icanhazip.com

136.118.191.229


In [ ]:
# 3. Run Streamlit in the background and expose it via localtunnel
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹

⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸your url is: https://hip-lies-clap.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://136.118.191.229:8501

